In [1]:
import sqlite3
import time
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import numpy as np

def solve_mTSP_kmeans_greedy(instance_id, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Instance information
    cursor.execute("SELECT nr_cities, nr_salesmen FROM instances WHERE instance_id = ?", (instance_id,))
    instance = cursor.fetchone()
    if not instance:
        print(f"Instance {instance_id} not found.")
        conn.close()
        return

    nr_cities, nr_salesmen = instance

    # City coordinates (excluding depot 0)
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ? AND city_id != 0", (instance_id,))
    cities = cursor.fetchall()

    if nr_salesmen > len(cities):
        print("Not enough cities for the number of salesmen.")
        conn.close()
        return

    city_ids = [city[0] for city in cities]
    coordinates = np.array([(city[1], city[2]) for city in cities])
    cities_dict = {city[0]: (city[1], city[2]) for city in cities}

    # KMeans clustering
    kmeans = KMeans(n_clusters=nr_salesmen, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(coordinates)

    # Initialize clusters
    clusters = {i: [] for i in range(nr_salesmen)}
    for i, label in enumerate(labels):
        clusters[label].append(city_ids[i])

    # Redistribute to avoid empty clusters
    empty_clusters = [k for k, v in clusters.items() if not v]
    while empty_clusters:
        for empty_cluster in empty_clusters:
            empty_centroid = kmeans.cluster_centers_[empty_cluster]
            # Choose from the largest cluster that has > 1 city
            largest_cluster = max((k for k in clusters if len(clusters[k]) > 1), key=lambda k: len(clusters[k]), default=None)
            if largest_cluster is None:
                continue
            candidate_cities = clusters[largest_cluster]
            candidate_coords = np.array([cities_dict[city_id] for city_id in candidate_cities])
            distances = cdist([empty_centroid], candidate_coords)[0]
            closest_idx = np.argmin(distances)
            city_to_move = candidate_cities.pop(closest_idx)
            clusters[empty_cluster].append(city_to_move)

        empty_clusters = [k for k, v in clusters.items() if not v]

    # Reintroduce depot (0)
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ?", (instance_id,))
    all_cities = cursor.fetchall()
    distance_matrix = [[
        ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for _, x2, y2 in all_cities
    ] for _, x1, y1 in all_cities]

    # Solve each cluster with greedy TSP
    start_time = time.time()
    routes = [[] for _ in range(nr_salesmen)]
    total_cost = 0
    distances = []

    for salesman_id in range(nr_salesmen):
        cluster_cities = clusters[salesman_id]
        route = [0]  
        visited = set(route)

        while cluster_cities:
            current_city = route[-1]
            nearest_city = min(cluster_cities,key=lambda city: distance_matrix[current_city][city])
            route.append(nearest_city)
            visited.add(nearest_city)
            cluster_cities.remove(nearest_city)

        route.append(0)  # Return to depot
        routes[salesman_id] = route

        route_distance = sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1))
        distances.append(route_distance)
        total_cost += route_distance

    end_time = time.time()
    time_taken = end_time - start_time

    distance_gap = max(distances) - min(distances) if distances else 0

    epsilon = 1e-6
    efficiency = total_cost / (time_taken + epsilon)

    # Save results
    cursor.execute("""
        INSERT OR REPLACE INTO algorithms (instance_id, strategy, total_cost, time_taken, distance_gap, efficiency)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (instance_id, "KMeans-Greedy", total_cost, time_taken, distance_gap, efficiency))

    cursor.execute("DELETE FROM routes WHERE instance_id = ? AND strategy = ?", (instance_id, "KMeans-Greedy"))
    for salesman_id, route in enumerate(routes):
        cursor.execute("""
            INSERT INTO routes (instance_id, strategy, salesman_id, route)
            VALUES (?, ?, ?, ?)
        """, (instance_id, "KMeans-Greedy", salesman_id, str(route)))

    conn.commit()
    conn.close()
    print(f"Instance {instance_id} solved using KMeans-Greedy.")

def solve_mTSP_kmeans_greedy_for_all_instances(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Select instances that have not been solved with KMeans-Greedy
    cursor.execute("""
        SELECT instance_id 
        FROM instances 
        WHERE instance_id NOT IN (
            SELECT instance_id 
            FROM algorithms 
            WHERE strategy = 'KMeans-Greedy'
        )
    """)
    # cursor.execute(""" SELECT instance_id FROM instances  """)
    instances = cursor.fetchall()

    if not instances:
        print("No unsolved instances found for KMeans-Greedy.")
        conn.close()
        return

    for (instance_id,) in instances:
        solve_mTSP_kmeans_greedy(instance_id, db_file)

    conn.close()


solve_mTSP_kmeans_greedy_for_all_instances(db_file="../validation_mTSP.sqlite3")

Instance 1 solved using KMeans-Greedy.
Instance 2 solved using KMeans-Greedy.
Instance 3 solved using KMeans-Greedy.
Instance 4 solved using KMeans-Greedy.
Instance 5 solved using KMeans-Greedy.
Instance 6 solved using KMeans-Greedy.
Instance 7 solved using KMeans-Greedy.
Instance 8 solved using KMeans-Greedy.
Instance 9 solved using KMeans-Greedy.
Instance 10 solved using KMeans-Greedy.
Instance 11 solved using KMeans-Greedy.
Instance 12 solved using KMeans-Greedy.
Instance 13 solved using KMeans-Greedy.
Instance 14 solved using KMeans-Greedy.
Instance 15 solved using KMeans-Greedy.
Instance 16 solved using KMeans-Greedy.
Instance 17 solved using KMeans-Greedy.
Instance 18 solved using KMeans-Greedy.
Instance 19 solved using KMeans-Greedy.
Instance 20 solved using KMeans-Greedy.
Instance 21 solved using KMeans-Greedy.
Instance 22 solved using KMeans-Greedy.
Instance 23 solved using KMeans-Greedy.
Instance 24 solved using KMeans-Greedy.
Instance 25 solved using KMeans-Greedy.
Instance 